In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 2

In [7]:
# exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1)
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [8]:
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 0.25*pi**2*torch.sin(pi/2*(1-x_norm))+(dimension -1)*pi/(2*x_norm)*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])+ u_temp**3

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.25*u_hat**4 - f_temp*u_hat)/x.size()[0]
    
    Nb = 200
    theta = 2*pi*(2*torch.rand(Nb)-1.0)
    y = torch.rand(Nb, dimension).cuda()
    y[:, 0] = torch.cos(theta)
    y[:, 1] = torch.sin(theta)
    part_2 = (torch.sum((model(y)-u_ex(y))**2)/y.size()[0])
    
    lambda1 = 200.0
    return part_1 + lambda1 * part_2 

In [10]:
Data_size = 2000
def Gendata():
    i=0
    notes = torch.tensor([])
    while notes.size()[0]< Data_size:
        temp_notes = 2*torch.rand(1000,dimension)-1
        index=torch.where(torch.norm(temp_notes,dim=1)<1)
        temp_notes = temp_notes[index]
        notes = torch.cat((notes,temp_notes),0)
    notes = notes[1:Data_size,:]
    return notes.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=2, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_2D_Dirichlet_nonlinear_lambda200.0.npy", error_save)
np.save("DRM_relative_error_2D_Dirichlet_nonlinear_lambda200.0.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(786.7537, device='cuda:0')
current relative error is:  tensor(2.4555, device='cuda:0')
current epoch is:  50
current loss is:  tensor(69.9704, device='cuda:0')
current relative error is:  tensor(1.4354, device='cuda:0')
current epoch is:  100
current loss is:  tensor(6.2480, device='cuda:0')
current relative error is:  tensor(1.1189, device='cuda:0')
current epoch is:  150
current loss is:  tensor(1.1617, device='cuda:0')
current relative error is:  tensor(0.9343, device='cuda:0')
current epoch is:  200
current loss is:  tensor(0.6355, device='cuda:0')
current relative error is:  tensor(0.8910, device='cuda:0')
current epoch is:  250
current loss is:  tensor(0.4525, device='cuda:0')
current relative error is:  tensor(0.8859, device='cuda:0')
current epoch is:  300
current loss is:  tensor(0.2331, device='cuda:0')
current relative error is:  tensor(0.8696, device='cuda:0')
current epoch is:  350
current loss is:  tensor(0.1087, device='cuda:

current epoch is:  3050
current loss is:  tensor(-0.9628, device='cuda:0')
current relative error is:  tensor(0.0955, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(-0.9730, device='cuda:0')
current relative error is:  tensor(0.0918, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(-0.9215, device='cuda:0')
current relative error is:  tensor(0.0884, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(-1.0021, device='cuda:0')
current relative error is:  tensor(0.0880, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(-0.9935, device='cuda:0')
current relative error is:  tensor(0.0842, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(-0.9317, device='cuda:0')
current relative error is:  tensor(0.0830, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(-0.9749, device='cuda:0')
current relative error is:  tensor(0.0800, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(-1.052

current epoch is:  6100
current loss is:  tensor(-0.9842, device='cuda:0')
current relative error is:  tensor(0.0347, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(-1.0277, device='cuda:0')
current relative error is:  tensor(0.0372, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(-0.9238, device='cuda:0')
current relative error is:  tensor(0.0355, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(-0.9723, device='cuda:0')
current relative error is:  tensor(0.0364, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(-0.9818, device='cuda:0')
current relative error is:  tensor(0.0339, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(-0.9742, device='cuda:0')
current relative error is:  tensor(0.0337, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(-1.0490, device='cuda:0')
current relative error is:  tensor(0.0339, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(-0.923

current epoch is:  9150
current loss is:  tensor(-0.9916, device='cuda:0')
current relative error is:  tensor(0.0218, device='cuda:0')
current epoch is:  9200
current loss is:  tensor(-0.9895, device='cuda:0')
current relative error is:  tensor(0.0228, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(-0.9835, device='cuda:0')
current relative error is:  tensor(0.0177, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(-0.9890, device='cuda:0')
current relative error is:  tensor(0.0214, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(-0.9987, device='cuda:0')
current relative error is:  tensor(0.0174, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(-0.9349, device='cuda:0')
current relative error is:  tensor(0.0168, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(-0.9744, device='cuda:0')
current relative error is:  tensor(0.0231, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(-1.041

current epoch is:  12200
current loss is:  tensor(-1.0576, device='cuda:0')
current relative error is:  tensor(0.0147, device='cuda:0')
current epoch is:  12250
current loss is:  tensor(-0.9799, device='cuda:0')
current relative error is:  tensor(0.0216, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(-1.0645, device='cuda:0')
current relative error is:  tensor(0.0186, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(-1.0400, device='cuda:0')
current relative error is:  tensor(0.0133, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(-0.9921, device='cuda:0')
current relative error is:  tensor(0.0128, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(-1.0017, device='cuda:0')
current relative error is:  tensor(0.0178, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(-1.0582, device='cuda:0')
current relative error is:  tensor(0.0125, device='cuda:0')
current epoch is:  12550
current loss is:  tenso

current epoch is:  15250
current loss is:  tensor(-0.9944, device='cuda:0')
current relative error is:  tensor(0.0107, device='cuda:0')
current epoch is:  15300
current loss is:  tensor(-0.9496, device='cuda:0')
current relative error is:  tensor(0.0063, device='cuda:0')
current epoch is:  15350
current loss is:  tensor(-1.0562, device='cuda:0')
current relative error is:  tensor(0.0116, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(-1.0513, device='cuda:0')
current relative error is:  tensor(0.0058, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(-1.0585, device='cuda:0')
current relative error is:  tensor(0.0165, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(-1.0471, device='cuda:0')
current relative error is:  tensor(0.0131, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(-1.0451, device='cuda:0')
current relative error is:  tensor(0.0122, device='cuda:0')
current epoch is:  15600
current loss is:  tenso

current epoch is:  18300
current loss is:  tensor(-0.9876, device='cuda:0')
current relative error is:  tensor(0.0093, device='cuda:0')
current epoch is:  18350
current loss is:  tensor(-0.9520, device='cuda:0')
current relative error is:  tensor(0.0106, device='cuda:0')
current epoch is:  18400
current loss is:  tensor(-1.0085, device='cuda:0')
current relative error is:  tensor(0.0055, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(-0.9820, device='cuda:0')
current relative error is:  tensor(0.0132, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(-1.0371, device='cuda:0')
current relative error is:  tensor(0.0071, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(-0.9874, device='cuda:0')
current relative error is:  tensor(0.0085, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(-1.0159, device='cuda:0')
current relative error is:  tensor(0.0141, device='cuda:0')
current epoch is:  18650
current loss is:  tenso